In [ ]:
import time, re, csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import InvalidSelectorException

import pandas as pd

driver = webdriver.Chrome('C:/Users/sooyeon/Downloads/chromedriver.exe')
driver.get(
    "https://movie.naver.com/movie/bi/mi/point.naver?code=99702"
)
time.sleep(2)

driver.find_element_by_css_selector("#content > div.article > div.mv_info_area > div.mv_info > h3 > a").text


In [8]:
driver.find_element_by_css_selector("#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6)").text

'다니엘 크레이그(제임스 본드), 라미 말렉(사핀), 라샤나 린치(노미)\n더보기'

In [9]:
driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[4])
result = []
for i in range(1,11):

    boxes = driver.find_elements_by_css_selector("body > div > div > div.score_result > ul > li")

    scores = []
    reviews= []
    ids = []
    dates = []
    for box in boxes:
        scores.append(box.find_element_by_css_selector("div.star_score > em").text)
        reviews.append(box.find_element_by_css_selector("div.score_reple > p").text)
        ids.append(box.find_element_by_css_selector("div.score_reple > dl > dt > em:nth-child(1)").text)
        dates.append(box.find_element_by_css_selector("div.score_reple > dl > dt > em:nth-child(2)").text)

    import pandas as pd

    df = pd.DataFrame({'id':ids,'score':scores,'date':dates,'review':reviews})

    df['movie_name'] = '007 노 타임 투 다이'
    result.append(df)
    driver.find_element_by_xpath('''//*[@id="pagerTagAnchor{}"]'''.format(str(i))).click()


# df.to_csv("tmp_.csv",sep = '|',index = None)

In [ ]:
# result : 데이터프레임으로 만들어질 리뷰 정보

In [10]:
pd.concat(result).reset_index(drop = True)

,id,score,date,review,movie_name
0,Fantasy(tbxo****),9,2021.09.29 18:52,"관람객 굿바이, 다니엘 크레이그",007 노 타임 투 다이
1,Debussy(kang****),1,2021.09.29 23:30,관람객 처음 10분은 정말 007 역대급 작품이 나오는 줄 알았다.. 근데 그게 끝...,007 노 타임 투 다이
2,묵(vlue****),2,2021.09.29 23:55,관람객 제임스본드를 그런식으로 퇴장시켜야만 했을까?별볼일없는 등장인물들에게 촬영분량...,007 노 타임 투 다이
3,milk****,7,2021.09.29 23:10,관람객 아니 감독양반 아무리 그래도 개간지 007 형님에게 도게자가 웬말이여..,007 노 타임 투 다이
4,로저페더러(alka****),1,2021.09.29 23:11,관람객 크레이그 007 중 최악의 작품. 후쿠나가가 007을 쓰레기로 만들었군요. ...,007 노 타임 투 다이
...,...,...,...,...,...
95,배리굿(sang****),7,2021.09.30 23:19,관람객 잘가 제임스본드... 잊지않을게,007 노 타임 투 다이
96,Assa(chld****),10,2021.09.29 18:55,"다니엘 크레이그가 보여 줄 수 있는 최고의 마지막, 덕분에 행복했어요..",007 노 타임 투 다이
97,김은영(key6****),10,2021.09.29 18:21,역시 007 실망시키지 않네요~ 시간 가는줄 모르고 봤어요~,007 노 타임 투 다이
98,스크랩헤븐(rty7****),6,2021.09.30 13:06,관람객 기대를 많이했는데 실망스러웠다,007 노 타임 투 다이


#

### 순서
* 연도별 페이지를 url로 설정 2019, 2020, 2021
* 영화목록은 한 페이지당 20개씩 보여짐
* 20개의 목록을 리스트에 담고 상세페이지로 들어간다.
* 영화 제목과 개봉일 수집
* 평점 확인 버튼을 누른다
* 리뷰목록은 한 페이지당 10개씩 보여짐
* 페이지네이션을 돌며 평점, 리뷰내용, 작성일을 담는다
* 20개의 목록에 있는 영화정보와 리뷰를 수집한 뒤 
* 다음  페이지로 넘어간다 

In [29]:
import pandas as pd
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome('C:/Users/sooyeon/Downloads/chromedriver.exe')

from urllib.request import urlopen
from bs4 import BeautifulSoup
link = []
result21=[]

# 해당 개봉년도의 페이지 순환
for page in range(1,38):
    url = f'https://movie.naver.com/movie/sdb/browsing/bmovie.naver?open=2021&page={page}'
    html = urlopen(url)
    soup = BeautifulSoup(html,'html.parser')
    ul = soup.find('div').find('ul',class_='directory_list')
    for a in ul.find_all('a'):
        link.append(a['href'])
    
    # 2021 페이지 접속   
    driver.get(url)
    
    # 시간 지연
    # time.sleep(2)

    # 영화 리스트 불러오기
    boxes = driver.find_elements_by_css_selector("#old_content > ul > li")
    link = []
    for box in boxes:
        link.append(box.find_element_by_css_selector('a').get_attribute('href'))

    # 링크 한줄 씩 반복문
    # 한 페이지당 해당하는 영화는 20개


    for i in range(0,19):
        # i번째 링크 접속하기
        driver.get(link[i])

        # 시간 지연 
        time.sleep(1)

        try:
            driver.find_element_by_xpath('//*[@id="movieEndTabMenu"]/li[5]/a').click()
            # 제목
            title= driver.find_element_by_css_selector("#content > div.article > div.mv_info_area > div.mv_info > h3 > a").text
            release_date = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(4)').text
            # iframe 조정 
            driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[4])

            # pagination 1~10
            for i in range(1,51):
            # 기준 : for i in range(1,11):
            
                # 리뷰 박스들
                boxes = driver.find_elements_by_css_selector("body > div > div > div.score_result > ul > li")
                
                # 평점
                score = []
                
                # 리뷰 내용
                content= []
                
                # 리뷰 작성일
                write_date = []
                
                # 리뷰 박스 한 개 내부에 포함된 요소들 append
                for box in boxes:
                    score.append(box.find_element_by_css_selector("div.star_score > em").text)
                    contents = box.find_element_by_css_selector("div.score_reple > p").text
                    # 관람객 평점인 경우 관람객이라는 단어가 포함됨
                    contents = contents.replace("\n","").replace('관람객','')
                    content.append(contents)
                    write_date.append(box.find_element_by_css_selector("div.score_reple > dl > dt > em:nth-child(2)").text)
                
                # 데이터 프레임 생성
                df = pd.DataFrame({'title':title,'score':score,'content':content,'write_date':write_date,'release_date':release_date})
                result21.append(df)
                
                # 다음 페이지로 이동
                driver.find_element_by_xpath('''//*[@id="pagerTagAnchor{}"]'''.format(str(i))).click()
        
        # 예외 처리
        except NoSuchElementException:
            continue
        except IndexError:
            continue
        except AttributeError:
            continue
        except ValueError:
            continue
        except InvalidSelectorException:
            continue
            
            
            # 맨 마지막 페이지의 영화는 20개가 넘지 않으므로 indexerror 발생 -> result에 영화목록은 잘 더해짐

IndexError: list index out of range

### 데이터프레임 생성

In [30]:
naver_review21 = pd.concat(result21).reset_index(drop=True)

In [31]:
naver_review21['source_site'] = 'naver'

### 데이터 구성
* title : 영화 제목
* score : 네티즌 평점
* content : 리뷰내용
* write_date : 리뷰작성일
* release_date : 영화 개봉일
* source_site : 리뷰출처 플랫폼 <- 맨 마지막에 열추가하는 방식

In [34]:
naver_review21

,title,score,content,write_date,release_date,source_site
0,007 노 타임 투 다이,9,"굿바이, 다니엘 크레이그",2021.09.29 18:52,2021 .09.29 개봉,naver
1,007 노 타임 투 다이,1,처음 10분은 정말 007 역대급 작품이 나오는 줄 알았다.. 근데 그게 끝이였다..,2021.09.29 23:30,2021 .09.29 개봉,naver
2,007 노 타임 투 다이,2,제임스본드를 그런식으로 퇴장시켜야만 했을까?별볼일없는 등장인물들에게 촬영분량을 주...,2021.09.29 23:55,2021 .09.29 개봉,naver
3,007 노 타임 투 다이,7,아니 감독양반 아무리 그래도 개간지 007 형님에게 도게자가 웬말이여..,2021.09.29 23:10,2021 .09.29 개봉,naver
4,007 노 타임 투 다이,1,크레이그 007 중 최악의 작품. 후쿠나가가 007을 쓰레기로 만들었군요. 이 작...,2021.09.29 23:11,2021 .09.29 개봉,naver
...,...,...,...,...,...,...
61592,흩어진 밤,10,스포일러가 포함된 감상평입니다. 감상평 보기,2021.06.25 20:59,2021 .06.24 개봉,naver
61593,흩어진 밤,10,I love you 180!,2021.06.25 07:54,2021 .06.24 개봉,naver
61594,흩어진 밤,10,어느 누구 한명의 잘못도 아닌데 모두가 흩어져야 하는 밤...,2021.06.24 11:43,2021 .06.24 개봉,naver
61595,흩어진 밤,1,"진짜 연기력도 그렇고 제작능력이며 스토리며 다 없어요,,,,감동도 재미도 ,,,",2021.06.27 16:10,2021 .06.24 개봉,naver


In [33]:
naver_review21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61597 entries, 0 to 61596
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         61597 non-null  object
 1   score         61597 non-null  object
 2   content       61597 non-null  object
 3   write_date    61597 non-null  object
 4   release_date  61597 non-null  object
 5   source_site   61597 non-null  object
dtypes: object(6)
memory usage: 2.8+ MB


In [35]:
naver_review21.groupby('title')['title'].count()

title
#아이엠히어              249
#위왓치유                40
007 노 타임 투 다이       500
117편의 러브레터           22
120시간: 로스트 인 아웃백     25
                   ... 
호스트: 접속금지            96
혼자 사는 사람들           500
화이트 온 화이트            25
화이트데이: 부서진 결계       149
흩어진 밤                41
Name: title, Length: 485, dtype: int64

#### 리뷰데이터 저장

In [32]:
naver_review21.to_csv('naver_review21.csv',index=False)

In [28]:
naver_review19

,title,score,content,write_date,release_date,source_site
0,0.0MHz,1,와 정말 무서운집 수준의 공포영화가 한국에 다시 나왔네요. 꼭 보러 가세요.착석하신...,2019.05.29 13:30,2019 .05.29 개봉,naver
1,0.0MHz,10,"내용 구성이나 연출, cg는 저예산 영화 정도의 수준이었지만 정은지가 연기를 잘한다",2019.05.29 10:58,2019 .05.29 개봉,naver
2,0.0MHz,2,상엽이 형은 어쩌다가 어릴때 죽었는지 다리 피흘리는 여고생 연관있을줄 알았는데 연관...,2019.05.29 20:15,2019 .05.29 개봉,naver
3,0.0MHz,10,정은지 연기 소름...,2019.05.29 12:21,2019 .05.29 개봉,naver
4,0.0MHz,6,일단 정은지는 완전 합격점. 최윤영도 좋았다. 여배우 둘 빼면 에휴...게다가 초반...,2019.05.29 11:34,2019 .05.29 개봉,naver
...,...,...,...,...,...,...
124667,"힘을 내요, 미스터 리",1,배우 연기력은 좋으나 스토리가 망.,2019.09.15 10:59,2019 .09.11 개봉,naver
124668,"힘을 내요, 미스터 리",5,진부한 신파 틀에박힌 코미디이번 엑시트의 성공을보고 배운게 없는걸까...,2019.09.14 12:05,2019 .09.11 개봉,naver
124669,"힘을 내요, 미스터 리",6,"딴 건 모르겠고, 음주운전이 별 거 아닌 것처럼 나온 서브 스토리에 어이가 없었다.",2019.09.13 20:49,2019 .09.11 개봉,naver
124670,"힘을 내요, 미스터 리",9,이유있는 코미디알고 우는 신파,2019.09.13 08:20,2019 .09.11 개봉,naver


In [26]:
naver_review19['source_site'] = 'naver'

In [25]:
naver_review19 = pd.concat(result19).reset_index(drop=True)

In [27]:
naver_review19.to_csv('naver_review2019.csv',index=False)

---

#### Test

In [49]:
import pandas as pd
# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome('C:/Users/sooyeon/Downloads/chromedriver.exe')

# 2019 페이지 접속
url = "https://movie.naver.com/movie/sdb/browsing/bmovie.naver?open=2019&page=1"
driver.get(url)

# 시간 지연
time.sleep(2)

# 영화 리스트 불러오기
boxes = driver.find_elements_by_css_selector("#old_content > ul > li")
link = []
for box in boxes:
    link.append(box.find_element_by_css_selector('a').get_attribute('href'))
    
# 링크 한줄 씩 반복문
# 한 페이지당 해당하는 영화는 20개

result = []
for i in range(0,20):
    # i번째 링크 접속하기
    driver.get(link[i])

    # 시간 지연 
    time.sleep(1)
    
    try:
        driver.find_element_by_xpath('//*[@id="movieEndTabMenu"]/li[5]/a').click()
        # 제목
        title= driver.find_element_by_css_selector("#content > div.article > div.mv_info_area > div.mv_info > h3 > a").text
        release_date = driver.find_element_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(4)').text
        driver.switch_to.frame(driver.find_elements_by_tag_name("iframe")[4])
        

        for i in range(1,3):
        # for i in range(1,11):
            boxes = driver.find_elements_by_css_selector("body > div > div > div.score_result > ul > li")

            score = []
            content= []
            write_date = []
            for box in boxes:
                score.append(box.find_element_by_css_selector("div.star_score > em").text)
                content.append(box.find_element_by_css_selector("div.score_reple > p").text)
                write_date.append(box.find_element_by_css_selector("div.score_reple > dl > dt > em:nth-child(2)").text)

            df = pd.DataFrame({'title':title,'score':score,'content':content,'write_date':write_date,'release_date':release_date})
            result.append(df)
            driver.find_element_by_xpath('''//*[@id="pagerTagAnchor{}"]'''.format(str(i))).click()

    except NoSuchElementException:
        continue
    except IndexError:
        continue

In [34]:
tmp_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26609 entries, 0 to 26608
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         26609 non-null  object
 1   score         26609 non-null  object
 2   content       26609 non-null  object
 3   write_date    26609 non-null  object
 4   release_date  26609 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


In [45]:
df

,score,date,review,title
0,9,2019.10.02 09:02,단순한 로맨스 영화가 아니다,가장 보통의 연애
1,10,2019.10.02 10:04,"관람객 연애와 직장, 모든것에 공감하며 미친듯이 웃다가- 현실적인 대사들에 뼈 후려...",가장 보통의 연애
2,10,2019.10.02 10:58,관람객 소오올찍히 그냥 저냥한 로코물인줄 알았는데 왜 공효진이 로코장인이라고 불리는...,가장 보통의 연애
3,10,2019.10.02 09:27,"재훈: ""남자랑 여자랑 같애?"" 선영: ""같지! 다르다고 배웠니 넌?"" ...",가장 보통의 연애
4,10,2019.10.02 10:40,"2012년 건축학개론을 보며 내 10대시절의 첫사랑의 향기를 느꼈다면, 2019년 ...",가장 보통의 연애
5,1,2019.10.03 21:39,영화가 술 처먹다 끝나네요 ㅋㅋㅋ,가장 보통의 연애
6,10,2019.10.02 09:06,진짜 재밌어요.. 꼭보세요김래원 공효진 조합은 정말 최고 조연분들 연기도 최고!! ...,가장 보통의 연애
7,10,2019.10.02 09:44,관람객 공효진이랑 김래원 케미 미쳤다 ㅋㅋㅋㅋ 입모양게임하는거 보고 웃겨 죽을뻔 ㅠ...,가장 보통의 연애
8,2,2019.10.04 10:00,"술로 시작해서 술로 끝나는 영화공감할수가 없었고 술을 안마시면 극복못하고, 술 마시...",가장 보통의 연애
9,1,2019.10.03 22:32,공감이 별로안되는 ...생각보다 별로였어요. 이게 무슨 주변에 있는 평범한 연애라는...,가장 보통의 연애
